# Chemical equilibrium with given element and charge amounts

We have seen that Reaktoro can perform chemical equilibrium calculations with a given initial chemical state in disequilibrium, e.g.,

In [ ]:
from reaktoro import *

# Initialize the NASA database
db = NasaDatabase("nasa-cea")

# Create a chemical system with only a gas phase
system = ChemicalSystem(db, GaseousPhase("CH4 O2 CO2 CO H2O H2"))

# Create an initial state for this chemical system
state = ChemicalState(system)
state.temperature(1000, "celsius")
state.pressure(100, "bar")
state.set("CH4", 1.0, "mol")
state.set("O2",  0.5, "mol")

# Create an equilibrium solver and equilibrates `state`
solver = EquilibriumSolver(system)
solver.solve(state)

print(state)

In the background of a `solver.solve(state)` function call, Reaktoro uses the *amounts of species* in the initial state to compute the *amounts of elements and charge* (further referred as *components*). These are needed in the underlying mass conservation equation solved along with the Gibbs energy minimization problem.

In some applications, the amounts of components of the system are given and should be used as an input to the chemical equilibrium calculation. For instance, in reactive transport modelling, at each mesh cell/node (or degree of freedom generated by discretization), you would have a vector `b` representing the concentrations of elements and charges at that specific point in space. Below, we demonstrate how this can be achieved to replicate the calculations performed above.

In [ ]:
import numpy

Ne = system.elements().size()      # the number of elements in the system

iC = system.elements().index("C")  # the index of component C
iH = system.elements().index("H")  # the index of component H
iO = system.elements().index("O")  # the index of component O
iZ = Ne                            # the index of component charge

b = numpy.zeros(Ne + 1)  # the array with explicit amounts of elements and charge

b[iC] = 1.0  # 1 mol of C
b[iH] = 4.0  # 4 moles of H
b[iO] = 1.0  # 1 mol of O
b[iZ] = 0.0  # 0 mol of charge

state = ChemicalState(system)
state.temperature(1000, "celsius")
state.pressure(100, "bar")

solver.solve(state, b)

print(state)

**TASK**: compare if the latter chemical state is indeed identical to equilibrium state computed before.